In [29]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# Downloaded this on local instead

# Character level language model!!

In [52]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [53]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115393


In [54]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [55]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [56]:
chars[0], chars[1]  # hehe new line and space. So below, everything that's a '0' in the encoded string is a new line :P

('\n', ' ')

In [57]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: given some string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))
# "tokenize" is just encoding the input sequence (words) into integers. lol.

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [58]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [59]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [60]:
block_size = 8 # context length
train_data[:block_size + 1] # +1 because there are 8 possible examples coming out of this (character level examples; see the below)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [61]:
x = train_data[:block_size]
y = train_data[1:block_size+1] # basically scroll the train data +1 right.
for t in range(block_size):
    context = x[:t+1] # btw when t=0, x[:0+1] = x[:1] = x[0] since it's exclusive
    target = y[t]
    print(f"when input is {context} the target: {target}")
x, y

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [62]:
torch.randint(len(data)-block_size, (batch_size,))

tensor([256195, 297194, 863950, 836453])

In [68]:
# Introducing batch dimensions
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split: str):
	# generate a small batch of data of inputs x and targets y
	data = train_data if split == 'train' else val_data
	ix = torch.randint(len(data)-block_size, (batch_size,)) # right now, 4 numbers (batch size #)'s generated between 0 and len(data)-block_size. Random offsets in the training set
	print(f'{ix.shape=}')
	x = torch.stack([data[i:i+block_size] for i in ix]) # get our 4 contexts and stack them
	y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # get the 4 respective targets (i.e shifted one right) and stack them
	return x, y # || x is (B,T), kinda..? B is batch size, T is time params/context length of that batch

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
	for t in range(block_size): # time dimension
		context = xb[b, :t+1]  # Remember, this gets the first *t* elements of the b row, since :x is exclusive.
		target = yb[b, t]
		print(f"when input is {context.tolist()} the target: {target}")
# 32 independent examples packed into the input xb and yb.

ix.shape=torch.Size([4])
inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
----
when input is [53] the target: 59
when input is [53, 59] the target: 6
when input is [53, 59, 6] the target: 1
when input is [53, 59, 6, 1] the target: 58
when input is [53, 59, 6, 1, 58] the target: 56
when input is [53, 59, 6, 1, 58, 56] the target: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target: 59
when input is [49] the target: 43
when input is [49, 43] the target: 43
when input is [49, 43, 43] the target: 54
when input is [49, 43, 43, 54] the target: 1
when input is [49, 43, 43, 54, 1] the target: 4

In [64]:
print(xb) # our input to the transformer

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])


In [65]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C), i.e (4, 8, 65). C is  # channels, vocab size in this case. So yeah, it converts all our inputs to have a 65 embedding thingy. || CHANNELS are like the 'information at each point'. That's the idea of it.

        if targets == None:
            loss = None # if we're not using targets (like for generation) just have none
        else:
            B,T,C =logits.shape # Convert so we can get the expect (B,C,T) for cross entropy
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # ||  Returns a 3D logits because we have no target, so it just embeds the index without doing anything else.

            # we only care about last time step so grab that
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) || In each batch dimension, we should have a single prediction (int) for what comes next.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) || Add new int prediction to the already existing context.
        return idx


m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape, loss)

# For generation
@torch.no_grad() # hey I added this :o
def generate_content(model, max_tokens=100):
	idx = torch.zeros((1,1), dtype=torch.long) # A little 1,1: 1 batch size, 1 time. Zero will be how we start the generation (i.e new line)
	generated_ints = model.generate(idx, max_new_tokens=max_tokens)[0].tolist() # index into 0 to unpluck the single batch dim that exists, so we get our timesteps indices thingies.
	print(decode(generated_ints)) # convert the generated ints to text
generate_content(m)

torch.Size([32, 65]) tensor(4.8948, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [66]:
# create a PyTorch optimizer || Oh sheeeet!! Okay, so in our makemore series, we always just updated our gradients with 'stochastic gradient descent' (would be optim.SGD(...)), but adamW is a big W up. It's better. gigachad.
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [67]:
# And our training loop!! Look how fucking small it is LOL
# We always went forward pass, backward pass, update gradients. 
# So using the optimiser, this is how you do it. Bruh the loop is 5 lines of code:
batch_size = 32
for _ in range(10000):
	# forward pass
	xb, yb = get_batch('train') 
	logits, loss = m(xb, yb)
	# backward pass 
	optimizer.zero_grad(set_to_none=True)
	loss.backward()
	# gradient descent/updating gradients with adamW now :o
	optimizer.step()

print(loss.item())

2.4782705307006836


His comments are just:
```py
batch_size = 32
for steps in range(100): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
```
Optimiser step is indeed our updating :P


In [ ]:
generate_content(m, max_tokens=500) # I mean... It's doing something!!


Mar is be eree t anverks m, t. whither, y ponofuthyfe br yoo wh o anse hinof ken he anu thor:
I

BUK:
SThais I tor qul s o thes t s is PSTabe'stouritavacitceif pomy indilagZSor buru hade himy are bled.
ILI pt ith ht.
THok, alle ag LAUTh LIOr ais proangre memford g fomy Seer id, aban y f ugg ue,
w; yotctomoorioryowas my cy hie,
Youeietharg wicajun
cousl fe s sopat, t:
Am-teas wnse be f CKI be itoupprsar.
Thicef s soyo hado'skince chit.

Thi'sangeff s I nd t e: wanenove s t Ineanand m,

Fonghinirm


## Okay, that's the basic bigram model!

Before we go onto letting the model see previous context more than just its immediately last point, let's move this stuff to a python script first (bigram.py)

#### The mathematical trick in self-attention

In [41]:
# consider the toy example
torch.manual_seed(1337)

B,T,C = 4,8,2 # batch, time, channels (channels is information at the timestep)
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [42]:
# We want x[b,t] = mean_{i<=t} x[b,i] || I.e to consider the previous context, we get the average of them (extremely lossy w.r.t information for now but fuck it concept is fine) and also our current context.

xbow = torch.zeros((B,T,C)) # bow is bag of words
for b in range(B):
	for t in range(T):
		xprev = x[b, :t+1] # (t, C) || t words, and ofc each word has its channel   || everything up to and including the t'th token on the given batch.
		xbow[b, t] = torch.mean(xprev, 0) # average out the time dimension (collapse it), so you get a little 1D C vector (and then stored in xbow)

In [43]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [44]:
xbow[0] # see how it's vertical average 

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [45]:
# So that's cool, but can we do this with matrices so it's a lot more efficient?
# 
# # toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
# a = torch.ones(3, 3)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)

b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')  
print(b)
print('--')
print('c=')
print(c)
# Comments below. But basically, now our a matrix can get us an average of all the above elements in a given b matrix.

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [46]:
""" 
First we have a = torch.ones(3), and so (3,3) * (3,2) -> (3,2) and each element of C is just it's a row dot producted by the b column. COol.
a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])

Guess what? If we do a lower triangle matrix with 
a = torch.tril(torch,ones(3,3)), 
when we multiply this by b we actually get the sum thingy lmao! Watch. Because it's just, yeah. It sums however many are above it (and itself) due to the dot product. Boom.

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.], # e.g 2 + 6 = 8, 7 + 4 = 11.
        [14., 16.]])

And then, if we wanted an average, we could just divide by the # of 1s in that given row. E.g for row 2 it'd be 8/2 = 4. 2 ones -> 2 numbers used, so we divide by two. We could also, as karpathy does,  just normalise the a rows, so then we'd be getting an average instead :o (expected value).

via 
a = torch.tril(torch,ones(3,3))
a = a / torch.sum(a, 1, keepdim=True) # And torch.sum is basically the 'counting of 1s' described above --> it's a tensor([[1.],[2.],[3.]]).
"""

" \nFirst we have a = torch.ones(3), and so (3,3) * (3,2) -> (3,2) and each element of C is just it's a row dot producted by the b column. COol.\na=\ntensor([[1., 1., 1.],\n        [1., 1., 1.],\n        [1., 1., 1.]])\n--\nb=\ntensor([[2., 7.],\n        [6., 4.],\n        [6., 5.]])\n--\nc=\ntensor([[14., 16.],\n        [14., 16.],\n        [14., 16.]])\n\nGuess what? If we do a lower triangle matrix with \na = torch.tril(torch,ones(3,3)), \nwhen we multiply this by b we actually get the sum thingy lmao! Watch. Because it's just, yeah. It sums however many are above it (and itself) due to the dot product. Boom.\n\na=\ntensor([[1., 0., 0.],\n        [1., 1., 0.],\n        [1., 1., 1.]])\n--\nb=\ntensor([[2., 7.],\n        [6., 4.],\n        [6., 5.]])\n--\nc=\ntensor([[ 2.,  7.],\n        [ 8., 11.], # e.g 2 + 6 = 8, 7 + 4 = 11.\n        [14., 16.]])\n\nAnd then, if we wanted an average, we could just divide by the # of 1s in that given row. E.g for row 2 it'd be 8/2 = 4. 2 ones -> 2 n


So going back to the non-matrixed' form (version 1): 
```py
xbow = torch.zeros((B,T,C)) # bow is bag of words
for b in range(B):
	for t in range(T):
		xprev = x[b, :t+1] # (t, C) || t words, and ofc each word has its channel   || everything up to and including the t'th token on the given batch.
		xbow[b, t] = torch.mean(xprev, 0) # average out the time dimension (collapse it), so you get a little 1D C vector (and then stored in xbow)
```


In [47]:
# version 2 
wei = torch.tril(torch.ones(T,T)) # weights -- a weighted aggregation. This way makes it so that our tokens will get info from only <= preceding tokens.
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T) @ (B,T,C) -> (B,T,T) @ (B,T,C) -> (B,T,C)
# Basically, it'll extend the BT matrix with another T, then multiply all the batches in parallel as a T,T @ T,C -> T,C.
torch.allclose(xbow, xbow2) # uhh idk why dw about this should be true 


False

In [48]:
xbow[0], xbow2[0] # Yeah it's fine

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [51]:
# version 3: we use Softmax. By setting the wei to 0 initially, we can configure them so it's a weighted aggregation, where not everything is equal. Some tokens might find other tokens more interesting
tril = torch.tril(torch.ones(T,T)) # lower ones matrix
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # replace anywhere where tril == 0 to -inf for exponentiation (normalising it pog): 0s, and -infs. || Forces you to be unable to 'interpret/use' tokens from the future at that current position (since e^-inf = 0).
wei = F.softmax(wei, dim=-1) # softmax on last dimension (i.e column direction or dim=1 here: exponent all the elements, then divide every element in a row by the sum of that row)
xbow3 = wei @ x
"""
The reason this is interesting is because we can now change the actual weights; i.e change how *much* the current token interacts (interaction strength/affinity) with certain past tokens.

And the -inf is so that we cannot communicate with tokens from the future since it'll get set to 0 in softmax via the exponentiation part.
"""
xbow[0], xbow3[0]
# torch.allclose(xbow, xbow3)


(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [28]:
# So, we've managed to make a weighted aggregation using matrices (efficient!), so tokens can look at previous tokens and have some interaction strength/affinity towards them.

# Crux of Self Attention!! Most important part of video to understand

We're gonna implement self-attention for a single individual 'head'. 

In [81]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, times, channels (the information at each token is 32dimensional)
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B,T,C) @ (C, head_size (16 here)) -> (B, T, 16)
q = query(x) # (B, T, 16). 
#No communication between tokens has happened yet. UNTIL NOOOW!
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)


# old code as before
tril = torch.tril(torch.ones(T,T)) # lower ones matrix
# wei = torch.zeros((T,T)) || Look how it's (T,T), but our self-attention new wei is (B,T,T)!! Our information is batch dependent now!!
wei = wei.masked_fill(tril == 0, float('-inf'))  # If this was removed, it'd be an encoder block. It's currently a decoder block, since it prevents you from looking/talking to future values :) 
wei = F.softmax(wei, dim=-1) 

v = value(x) # (4, 8, 32) @ (32, 16)
out = wei @ v

# out = wei @ x # (4, 8, 32)

out.shape

torch.Size([4, 8, 16])

In [79]:
print(wei.shape) #wei
## See how for each batch (for the full wei), the TxT table is different? 
# And also, for a given batch like wei[0]
wei[0] # Our probabilities aren't equal anymore (still sum to 1 ofc).  But now, we can have different embeddings communicate differently to others, with some having higher affinity towards others.

torch.Size([4, 8, 8])


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

## Notes about attention
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

#### Why we wanna do 1/sqrt(head_size) 

In [84]:

k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) # * head_size**-0.5


<function Tensor.var>

In [88]:
k.var(), q.var(), wei.var() #  Weight variance is massive (if the head size part is commented out), which means large number difference in the tensor right? And the thing about this, is it makes the softmax function converge to a one-hot the larger the differences are. See the example below.count

""" 
And the reason we want wei to be var 1, is we want the softmax to be diffuse -- similar probabilities over all outcomes, instead of incorrectly confidently selecting one output.
"""

(tensor(0.9006), tensor(1.0037), tensor(15.9317))

In [94]:
torch.softmax(torch.tensor([0.1, 0.2, -0.1, -0.3, 0.5]), dim=-1) # last dim (i.e only dim here lol)

tensor([0.1966, 0.2173, 0.1610, 0.1318, 0.2933])

In [96]:
torch.softmax(torch.tensor([0.1, 0.2, -0.1, -0.3, 0.5]) * 8, dim=-1) # Only differences is the *8, i.e the values have bigger gaps/larger variance.

# See how the last value (largest one) looks a lot closer to 1 probability, and the rest are basically yeeted away. Since the exponentiation is so much larger :o

# It's like any node (token) aggreggates information from one single node, instead of everything else. not good, especially at initialisation!

tensor([0.0357, 0.0795, 0.0072, 0.0015, 0.8761])

In [116]:
# Random stuff
test = nn.Embedding(10,3)

testtens = torch.tensor([1,4])
test(testtens)

tensor([[-1.0978,  0.5847, -0.2079],
        [ 1.0773,  0.1480,  0.6040]], grad_fn=<EmbeddingBackward0>)

### Layernorming
Okay we did a fair bit in the v2.py, so yeah. 

In [117]:
class LayerNorm1d: # (used to be BatchNorm1d). Literally the only change was the dim we meaned/var over (from 0 to 1). And since we're normalising column-wise (i.e each row), we can get rid of the buffers for inference times.
  
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [118]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs. 
#No longer (0, 1) now that we've normalised the layer instead of the batch.

(tensor(0.1469), tensor(0.8803))

In [119]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features. 

(tensor(-9.5367e-09), tensor(1.0000))